In [2]:
from gtts import gTTS
from IPython.display import Audio
import speech_recognition as sr
import io
import pyaudio
import simpleaudio as sa
import requests
import openai
import os
import time
from pydub import AudioSegment

In [8]:
def listen_to_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Fale algo...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("Você disse: " + text)
            return text
        except sr.UnknownValueError:
            print("Não entendi o áudio")
            return None
        except sr.RequestError as e:
            print("Erro de requisição; {0}".format(e))
            return None

def get_response(text):
      
    from openai import OpenAI
    api_key = 'SUA-API-OPENAI'
    client = OpenAI(api_key=api_key)

    completion = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": text}
    ],
    max_tokens=150
    )

    message = completion.choices[0].message.content
    # definindo parâmetros do gtts
    language = "en"


    # Instanciando o modelo na variável
    speech =gTTS(text=message,lang=language,slow=False)

    #  Criand um buffer de memória (uma espécie de arquivo temporário na memória).
    audio_buffer = io.BytesIO()

    # Escreve o áudio gerado pelo gTTS no buffer de memória.
    speech.write_to_fp(audio_buffer)

    # Reposiciona o cursor no início do buffer de memória, preparando-o para leitura.
    audio_buffer.seek(0)

   
    return Audio(audio_buffer.read(), autoplay=True)

def estimate_audio_duration(text, words_per_minute=160):
    word_count = len(text.split())
    minutes = word_count / words_per_minute
    return minutes * 60  # convertendo minutos em segundos

In [ ]:
def main():
    while True:
        spoken_text = listen_to_speech()
        if spoken_text:
            audio_response = get_response(spoken_text)
            
            
            
             # Estimar a duração do áudio
            duration = estimate_audio_duration(spoken_text) * 4
            print(duration)
            display(audio_response)
            time.sleep(duration)
            
            if spoken_text.lower() == "stop":
                print("Conversa terminada")
                break
        
                       

if __name__ == "__main__":
    main()